In [1]:
from proj1_helpers import *
from functions import *
import pandas as pd
import numpy as np
y, tX, ids = load_csv_data("data/train.csv")
y[np.where(y == -1)] = 0
tX = rearrange_continuous_categorical_features(tX)

In [2]:
categories = tX[:, -1]
zeros_index = np.where(categories == 0)[0]
one_index = np.where(categories == 1)[0]
two_index = np.where(categories == 2)[0]
three_index = np.where(categories == 3)[0]

zeros = tX[zeros_index, :]
null_var_index = np.where(np.std(zeros, axis=0) == 0)[0]
zeros = np.delete(zeros, null_var_index, axis=1)
zeros[np.where(zeros == -999)] = np.nan
zeros = median_imputation(zeros)
y_zero = y[zeros_index]

ones = tX[one_index, :]
null_var_index = np.where(np.std(ones, axis=0) == 0)[0]
ones = np.delete(ones, null_var_index, axis=1)
ones[np.where(ones == -999)] = np.nan
ones = median_imputation(ones)
y_one = y[one_index]

two = tX[two_index, :]
null_var_index = np.where(np.std(two, axis=0) == 0)[0]
two = np.delete(two, null_var_index, axis=1)
two[np.where(two == -999)] = np.nan
two = median_imputation(two)
y_two = y[two_index]

three = tX[three_index, :]
null_var_index = np.where(np.std(three, axis=0) == 0)[0]
three = np.delete(three, null_var_index, axis=1)
three[np.where(three == -999)] = np.nan
three = median_imputation(three)
y_three = y[three_index]

In [3]:
def find_separator(y, x, max_p=1e-14, nb_sep=10, max_iters=1000, max_categories=10):
    cat_index = is_cat(x, max_categories)
    categorical_variables = x[:, cat_index]
    continuous_variables = x[:, ~cat_index]

    indexes = list()
    counter = 0
    dim2 = x.shape[1]

    for i in range(dim2):
        for j in range(dim2):
            if i>=j:
                continue
            for k in range(dim2):
                if j>k:
                    continue
                candidate = x[:,i]*x[:,j]*x[:,k]
                res = gaussian_test(y, candidate)
                print(f"Current : {res}                                                   \r", end="")
                counter += 1
                if res < max_p:
                    indexes.append([i,j,k])
                if len(indexes) >= nb_sep or max_iters == counter:
                    break
            if len(indexes) >= nb_sep or max_iters == counter:
                break
        if len(indexes) >= nb_sep or max_iters == counter:
                break

    return indexes

In [4]:
asd, __ = orthogonal_basis(ones)
new_dim = find_separator(y_one, asd, max_p = 1e-2, nb_sep=40, max_iters=1000)

In [5]:
res = np.array([asd[:, new_dim[i][0]] * asd[:, new_dim[i][1]] * asd[:, new_dim[i][2]] for i in range(len(new_dim))]).T

In [6]:
import pandas as pd
pd.DataFrame(np.corrcoef(res, rowvar=False))

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,1.000000,-0.063829,-0.073635,0.195732,-0.263848,0.088482,-0.186397,0.194804,-0.317681,3.230771e-01,...,-0.061388,0.259124,0.044414,0.038438,0.162137,-0.009305,0.014233,0.303324,-0.184084,2.579508e-03
1,-0.063829,1.000000,0.234914,0.140349,0.193575,0.082631,0.151999,-0.414805,0.280125,4.076249e-01,...,-0.091134,-0.276668,0.008902,-0.085710,-0.070076,-0.021072,0.022699,-0.098762,0.220616,-8.503214e-02
2,-0.073635,0.234914,1.000000,-0.184085,-0.219569,0.045361,0.294704,-0.374030,0.086945,6.764032e-02,...,0.101294,-0.062074,0.055525,-0.132897,-0.009713,-0.012334,-0.015927,-0.069030,0.173636,3.245156e-02
3,0.195732,0.140349,-0.184085,1.000000,-0.117592,-0.049351,-0.333351,0.199034,-0.297480,5.702730e-01,...,-0.510314,0.079689,-0.013259,0.106694,0.117162,0.029879,0.048363,0.258274,-0.132724,-1.295457e-02
4,-0.263848,0.193575,-0.219569,-0.117592,1.000000,0.051110,0.171885,0.092471,0.326073,1.224911e-01,...,0.047656,0.029575,-0.080473,0.459874,-0.558484,-0.018342,0.009139,-0.101196,-0.090880,-2.670371e-02
5,0.088482,0.082631,0.045361,-0.049351,0.051110,1.000000,-0.042424,0.002113,-0.029817,8.613741e-02,...,0.051319,-0.050647,-0.441521,-0.034886,0.023960,0.520954,-0.761970,0.284169,0.106544,5.256594e-02
6,-0.186397,0.151999,0.294704,-0.333351,0.171885,-0.042424,1.000000,-0.343423,0.050837,-3.964256e-02,...,-0.010968,-0.121196,-0.013066,-0.061293,-0.037726,0.045914,0.031344,-0.541969,0.445111,-5.455420e-01
7,0.194804,-0.414805,-0.374030,0.199034,0.092471,0.002113,-0.343423,1.000000,-0.111108,4.710247e-03,...,-0.018566,0.314088,-0.061349,0.223345,-0.021753,0.085722,-0.010896,0.204936,-0.411905,5.229113e-02
8,-0.317681,0.280125,0.086945,-0.297480,0.326073,-0.029817,0.050837,-0.111108,1.000000,-2.314273e-01,...,0.215912,-0.187810,0.027239,-0.004416,-0.360236,-0.001634,0.016796,-0.222262,0.076537,-2.153681e-02
9,0.323077,0.407625,0.067640,0.570273,0.122491,0.086137,-0.039643,0.004710,-0.231427,1.000000e+00,...,-0.221749,0.080724,-0.087175,0.128910,0.126954,0.020291,0.019820,0.187970,-0.052256,-9.299836e-07


In [7]:
means_1 = list()
medians_1 = list()
stds_1 = list()
for i in range(1, 21):
    print("\n")
    print("*****************")
    print("\n")
    print(i)
    tx, tosolve_tx = orthogonal_basis(ones)
    tx = process_data(x = tx, degree=i, pairwise=False, bias=False)
    tx = np.c_[tx, np.array([tx[:, new_dim[i][0]] * tx[:, new_dim[i][1]] * tx[:, new_dim[i][2]] for i in range(len(new_dim))]).T]
    tx, _, __ = gaussian_scaling(tx)
    tx = process_data(x = tx, degree=0, pairwise=False, bias=True)
    acc, m, md, std_ = cross_validation(y_one, tx, k_fold=5)
    means_1.append(m)
    medians_1.append(md)
    stds_1.append(std_)



*****************


1

CV 1/5 --- Validation accuracy : 74.503%  


C:\Users\Antek\GitHub\HBP\scripts\functions.py:242: RuntimeWarning: divide by zero encountered in log
  (1 - y).T.dot(np.log(1 - proba))) + 2*lambda_*np.linalg.norm(w)


CV 2/5 --- Validation accuracy : 74.729%
CV 3/5 --- Validation accuracy : 75.013%  
CV 4/5 --- Validation accuracy : 74.71%   
CV 5/5 --- Validation accuracy : 74.529%  

 
 Mean validation accuracy = 0.7469693061645601 
 Median validation accuracy = 0.7470982718596852 
 Std validation accuracy = 0.0018252177264577482


*****************


2

Polynomial augmentation progress : 100.0%                 
CV 1/5 --- Validation accuracy : 77.966%  
CV 2/5 --- Validation accuracy : 78.25%   
CV 3/5 --- Validation accuracy : 78.082%  
CV 4/5 --- Validation accuracy : 78.366%  
CV 5/5 --- Validation accuracy : 78.372%  

 
 Mean validation accuracy = 0.7820737683776116 
 Median validation accuracy = 0.7824993551715244 
 Std validation accuracy = 0.0016002143274950805


*****************


3

Polynomial augmentation progress : 100.0%                 
CV 1/5 --- Validation accuracy : 78.392%  
CV 2/5 --- Validation accuracy : 78.746%  
CV 3/5 --- Validation accuracy : 78.837%  
CV 4/5 --- Validat

C:\Users\Antek\GitHub\HBP\scripts\functions.py:235: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-x))


C:\Users\Antek\GitHub\HBP\scripts\functions.py:241: RuntimeWarning: divide by zero encountered in log
  loss = (-1/len(y))*(y.T.dot(np.log(proba)) +


CV 1/5 --- Validation accuracy : 78.882%
CV 2/5 --- Validation accuracy : 78.856%  
CV 3/5 --- Validation accuracy : 79.146%  
CV 4/5 --- Validation accuracy : 79.237%  
CV 5/5 --- Validation accuracy : 79.398%  

 
 Mean validation accuracy = 0.7910368841888058 
 Median validation accuracy = 0.7914624709827186 
 Std validation accuracy = 0.0020805488444400025


*****************


5

Polynomial augmentation progress : 100.0%                 
CV 1/5 --- Validation accuracy : 78.888%  
CV 2/5 --- Validation accuracy : 79.062%  
CV 3/5 --- Validation accuracy : 79.043%  
CV 4/5 --- Validation accuracy : 79.062%  
CV 5/5 --- Validation accuracy : 79.243%  

 
 Mean validation accuracy = 0.7905984008253805 
 Median validation accuracy = 0.7906241939644054 
 Std validation accuracy = 0.001124740578303628


*****************


6

Polynomial augmentation progress : 100.0%                 
CV 1/5 --- Validation accuracy : 78.94%   
CV 2/5 --- Validation accuracy : 79.095%  
CV 3/5 --- Validati

In [ ]:
for i in range(ones.shape[1]):
    print(gaussian_test(y_one, ones[:,i]))

In [ ]:
for i in range(two.shape[1]):
    print(gaussian_test(y_two, two[:,i]))